In [23]:
from datasets import load_dataset
import os

path_to_dataset = "../../datasets/KIND/SLIMER"
train_data = load_dataset("json", data_files=os.path.join(path_to_dataset, 'train.jsonl'))['train']
validation_data = load_dataset("json", data_files=os.path.join(path_to_dataset, 'validation.jsonl'))['train']
test_data = load_dataset("json", data_files=os.path.join(path_to_dataset, 'test.jsonl'))['train']

Using custom data configuration default-ce9b5fc5380027d8
Found cached dataset json (/Users/andrew/.cache/huggingface/datasets/json/default-ce9b5fc5380027d8/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-eb84874c7e8b3dd0
Found cached dataset json (/Users/andrew/.cache/huggingface/datasets/json/default-eb84874c7e8b3dd0/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-612a535ed258c50e
Found cached dataset json (/Users/andrew/.cache/huggingface/datasets/json/default-612a535ed258c50e/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
print(train_data)
print(validation_data)
print(test_data)

Dataset({
    features: ['doc_tag_pairID', 'input', 'tagName', 'instruction', 'output'],
    num_rows: 82446
})
Dataset({
    features: ['doc_tag_pairID', 'input', 'tagName', 'instruction', 'output'],
    num_rows: 14301
})
Dataset({
    features: ['doc_tag_pairID', 'input', 'tagName', 'instruction', 'output'],
    num_rows: 12369
})


In [25]:
print(train_data[0])

{'doc_tag_pairID': 'ADG:train:0:0', 'input': 'Il ministro degli Esteri al commissario capo della Commissione alleata', 'tagName': 'LOC', 'instruction': "Estrai tutte le entità di tipo LOCALITÀ dal testo che hai letto. Ti vengono fornite una DEFINIZIONE e alcune LINEE GUIDA.\nDEFINIZIONE: LOCALITÀ denota nomi propri di luoghi geografici, comprendendo città, paesi, stati, regioni, continenti, punti di interesse naturale, e indirizzi specifici.\nLINEE GUIDA: Assicurati di non confondere i nomi di luoghi con nomi di persone, organizzazioni o altre entità. Ad esempio, 'Washington', potrebbe riferirsi alla città di Washington D.C. o al presidente George Washington, quindi considera attentamente il contesto. Escludi nomi di periodi storici, eventi o concetti astratti che non rappresentano luoghi fisici. Ad esempio, 'nel Rinascimento' è un periodo storico, non una località geografica.\nRestituisci una lista JSON di istanze di questo tipo. Restituisci una lista vuota se non sono presenti istanz

In [11]:
positive_samples = 0
for sample in train_data:
    if sample['output'] != '[]':
        positive_samples += 1
        
print(positive_samples)

15092


In [4]:
import sys
sys.path.append('/Users/andrew/ExpertAI/SLIMER_IT')
from KIND import KIND

In [5]:
path_to_BIO = '../../datasets/KIND/evalita-2023'

dataset_KIND_manager = KIND(path_to_BIO,
                            path_to_templates='../templates',
                            SLIMER_prompter_name='SLIMER_instruction_it',
                            path_to_DeG='../def_and_guidelines/KIND.jsonl')


Using prompt template SLIMER_instruction_it: SLIMER instruction - Italian



In [6]:
dataset_n_per_tag = dataset_KIND_manager.get_Npos_Mneg_per_topXtags(1000, 1000)

In [8]:
print(dataset_n_per_tag)

DatasetDict({
    train: Dataset({
        features: ['doc_tag_pairID', 'input', 'tagName', 'instruction', 'output'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['doc_tag_pairID', 'input', 'tagName', 'instruction', 'output'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['doc_tag_pairID', 'input', 'tagName', 'instruction', 'output'],
        num_rows: 12369
    })
})


In [12]:
print(dataset_n_per_tag['train'][:10])

{'doc_tag_pairID': ['WN:train:7893:1', 'FIC:train:9439:1', 'WN:train:3630:0', 'WN:train:5347:1', 'ADG:train:2192:0', 'WN:train:8483:1', 'FIC:train:3383:1', 'FIC:train:9379:0', 'WN:train:2537:0', 'WN:train:6021:1'], 'input': ["La manifestazione dell' estrema destra era programmata da tempo e da tempo circolavano voci di una contromanifestazione di protesta organizzata da alcuni dei così detti centri sociali .", 'Esse mandarono un regalo al nipotino , ma non scrissero alla madre .', 'Un terribile incidente stradale si è verificato nella prima mattinata di ieri nei pressi della cittadina francese di Puisseguin , nel dipartimento della Gironda , nel sud-ovest del Paese :', "Johnson , che era risultato positivo al virus SARS-CoV - 2 nel pomeriggio di domenica , era stato ricoverato in terapia intensiva ad inizio settimana visto l' aggravarsi della sua condizione clinica .", "Anche qui l' organizzazione , se è vitale e partecipe , come ha partecipato , all' attività internazionale del Movime

In [17]:
samples_per_subdataset = {}
for sample in dataset_n_per_tag['train']:
    subdataset = sample['doc_tag_pairID'].split(':')[0]
    if subdataset not in samples_per_subdataset:
        samples_per_subdataset[subdataset] = 0
    else:
        samples_per_subdataset[subdataset] += 1
print(samples_per_subdataset)

{'WN': 2958, 'FIC': 1897, 'ADG': 1142}


In [22]:
sampled_inputs = {}
for sample in dataset_n_per_tag['train']:
    input_id = sample['doc_tag_pairID'].split(':')[-2]
    if input_id not in sampled_inputs:
        sampled_inputs[input_id] = 0
    else:
        sampled_inputs[input_id] += 1
print(sorted(sampled_inputs.items(), key=lambda x:x[1], reverse=True))

[('2777', 4), ('10229', 3), ('61', 3), ('479', 3), ('2427', 3), ('921', 3), ('3302', 3), ('1263', 3), ('1415', 3), ('1644', 3), ('2669', 3), ('8711', 3), ('4264', 3), ('3711', 3), ('1590', 3), ('2860', 3), ('707', 3), ('4183', 3), ('1154', 3), ('1831', 3), ('1265', 3), ('2945', 3), ('9433', 2), ('6889', 2), ('1488', 2), ('3622', 2), ('4533', 2), ('5294', 2), ('67', 2), ('1983', 2), ('6603', 2), ('5381', 2), ('3810', 2), ('10727', 2), ('3102', 2), ('7203', 2), ('2803', 2), ('4050', 2), ('2470', 2), ('291', 2), ('10594', 2), ('752', 2), ('4711', 2), ('9134', 2), ('4874', 2), ('2542', 2), ('1024', 2), ('1548', 2), ('3662', 2), ('3207', 2), ('3750', 2), ('3505', 2), ('6166', 2), ('1180', 2), ('755', 2), ('8493', 2), ('5697', 2), ('6135', 2), ('7281', 2), ('1537', 2), ('1762', 2), ('2224', 2), ('3063', 2), ('5835', 2), ('3963', 2), ('967', 2), ('4392', 2), ('4776', 2), ('2351', 2), ('3958', 2), ('3278', 2), ('4299', 2), ('3943', 2), ('1428', 2), ('4720', 2), ('684', 2), ('201', 2), ('6585',

In [10]:
def filter_by_prefix(dataset, prefix):
    def filter_function(example):
        id_prefix = example["doc_tag_pairID"].split(":")[0]
        return id_prefix == prefix

    return dataset.filter(filter_function)

In [11]:
desired_prefix = "WN"

# Apply the filter with the desired prefix
filtered_dataset = filter_by_prefix(dataset_n_per_tag['train'], desired_prefix)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [12]:
print(filtered_dataset[0:10])

{'doc_tag_pairID': ['WN:train:3675:1', 'WN:train:10900:2', 'WN:train:5034:1', 'WN:train:1410:0', 'WN:train:9287:2', 'WN:train:6104:1', 'WN:train:1317:0', 'WN:train:10391:2', 'WN:train:8856:1', 'WN:train:10246:1'], 'input': ["È morto l' allenatore di pallavolo Vjačeslav Platonov", 'Il disegno di legge è passato alla Camera dei Rappresentanti con 86 favorevoli e 63 contrari , mentre era passato in Senato una settimana prima .', "Di Battista ha rinnovato i suoi auspici che non si trovi alcun accordo per portare Forza italia al governo , criticando il berlusconismo come fenomeno politico e ha commentato sui possibili motivi dietro le quinte che potrebbero spingere il leader leghista a mantenere l' alleanza con Berlusconi , citando in particolare le fideiussioni che legherebbero l' ex cavaliere al Carroccio .", 'Il matrimonio civile con Stefania Lillo , redattrice di alcuni programmi di Radio Due , si è tenuto nella chiesa sconsacrata nei pressi delle Terme di Caracalla , a Roma , ed è stat